# 30cappa sul percorso stradale 
## analisi valida per il Trentino sull'ordinanza numero 69 del 2020

In [1]:
import rtree
import pandas as pd
import numpy as np
import geopandas as gpd
import openrouteservice
import matplotlib.pyplot as plt
import time
import json
import os, glob
from shapely import geometry
pd.options.mode.chained_assignment = None

# import dei dati
- i confini comunali del Trentino derivano dai dati ISTAT del 2020
- il grafo stradale deriva da OpenStreetMap

In [2]:
comuni_trentini_url = "https://github.com/napo/pat30k/raw/main/data/comuni_trentini.gpkg"
roads_url ="https://github.com/napo/pat30k/raw/main/data/trentino_roads_osm.gpkg"

In [3]:
comuni_trentini = gpd.read_file(comuni_trentini_url)

creazione dei confini provinciali

In [4]:
confini_pat = comuni_trentini.dissolve()
confine_trento = comuni_trentini[comuni_trentini.PRO_COM_T=='022205']
confine_trento = confine_trento.to_crs(epsg=4326)
confine_trento.to_file("trento.geojson",driver="GeoJSON")

## preparazione dei dati
l'ordinanza fa riferimento ai comuni fino a 6000 abitanti (dati ISTAT al 01/01/2020)

In [5]:
comuni_under6000 = comuni_trentini[comuni_trentini.POPOLAZIONE <= 6000]

i dati ISTAT sono proiettati in UTM32N (WGS84)

In [6]:
comuni_under6000 = comuni_under6000.to_crs(epsg=4326)

i dati OpenStreetMap sono utilizzati per individuare i punti di accesso sulconfine comunale

In [7]:
roads_osm_pat = gpd.read_file(roads_url)

si scelgono i punti di accesso su strade percorribili da auto: è necessario quindi evitare percorsi di montagna, strade abbandoante, sentieri ecc...

In [8]:
exclude_highway=['footway','bridleway','rest_area','abandoned','ohm:military:Trench','steps','via_ferrata','elevator','proposed','disused','platform','raceway','path','construction','pedestrian','cycleway','track']

In [9]:
roads_osm_pat = roads_osm_pat[~roads_osm_pat.isin(exclude_highway)]

alcune strade sembrano non essere classificate, quindi è opportuno eliminarle

In [10]:
roads_osm_pat = roads_osm_pat[roads_osm_pat.highway.notnull()]

# Calcolo delle isodistanze a 30km
- il calcolo è fatto usando OpenRouteService su una istanza personale
- i punti scelti per il calcolo dell'isodistanza sono quelli del punto di intersezione fra il confine e le strade

## uso di OpenRouteService
- per l'installazione di OpenRouteService seguire la documentazione ufficiale
- i dati scelti per il calcolo sono basati a partire dal confine della Provincia autonoma di Trento con un buffer di 40km con una estrazione da Geofabrik

**getPolygon(pp)**: 
- calcolo dell'istodistanza di 30km a partire da un punto (pp) utilizzando l'istanza in locale  (http://locahost:8080/ors) di OpenRouteService

In [11]:
def getPolygon(pp):
    params_iso = {'profile': 'driving-car',
              'range': [30000], 
              'range_type': "distance",
              'interval': 30000
             }
    orsclient = openrouteservice.Client(base_url='http://localhost:8080/ors')
    params_iso['locations'] = [pp]
    out = None
    polygon = None
    try:
        out = orsclient.isochrones(**params_iso)
    except Exception as e:
        print("errore nelle coordinate ")
        print(pp)
        print(e)
        pass
    if out is not None:
        points_poly = out["features"][0]['geometry']['coordinates']
        polygon = geometry.Polygon(points_poly[0])
    return(polygon)

**comunearea30km(idcomune,strade_union)**: q
- estrazione dei confini del singolo comune (a partire dal codice ISTAT)
- trasformazioen dei poligoni in linee
- individuazione dei punti di accesso al confine comunale intersecando con l'insieme delle strade **strade_unione** = la geometria MultiLineString che contiene tutte le strade calcolate con la funzione **roads_osm_pat.unary_union**

In [12]:
def comunearea30km(idcomune,strade_union):
  comune = comuni_under6000[comuni_under6000.PRO_COM_T==idcomune]
  poligoni_comune = comune.explode()
  linee_comune = poligoni_comune.exterior
  linee_comune = gpd.GeoDataFrame(geometry=linee_comune,crs="EPSG:4326")
  accessi_comune = linee_comune.unary_union.intersection(strade_union)
  punti_accesso_comune = []
  polygons = []
  k = 0
  if (accessi_comune.geom_type == 'Point'):
    p = [accessi_comune.x,accessi_comune.y]
    punti_accesso_comune.append(p)
    polygon = getPolygon(p)
    if (polygon is not None):
      polygons.append(polygon)
      k += 1
  else:
    for point in accessi_comune:
      p = [point.x,point.y]
      punti_accesso_comune.append(p)
      polygon = getPolygon(p)
      if (polygon is not None):
        polygons.append(polygon)
        k += 1
  area = gpd.GeoDataFrame(geometry=polygons,crs="EPSG:4326")
  return(area)

## calcolo delle isodistanze
applicazione delle due funzioni disegnate sopra dall'elenco dei codici ISTAT dei comuni


### note 
- le aree calcolate per ogni punto di accesso vengono unite in una sola geometria (**dissolve**)
- vengono segnalati eventuali errori nel caso in cui OpenRouteService non sia in grado di calcolare l'isodistanza
- ogni area viene salvata nella directory "**data**"

In [21]:
strade_union = roads_osm_pat.unary_union
comuni = comuni_under6000.PRO_COM_T.unique()

In [22]:
for idcomune in comuni:
  print("calcolo della isodistanza per ... " + idcomune)
  comune = comuni_under6000[comuni_under6000.PRO_COM_T == idcomune]
  filename = idcomune + ".geojson"
  area = comunearea30km(idcomune,strade_union)
  area.dissolve().to_file("data" + os.sep + filename,driver="GeoJSON")

calcolo della isodistanza per ... 022009


In [23]:
os.chdir('data')
geojson_files = glob.glob("*.geojson")
os.chdir("..")
areasdata = "data" + os.sep + "areas30km"
if not os.path.exists(areasdata):
    os.makedirs(areasdata)

## applicazione delle regole dell'ordinanza
- **ci si può muovere alĺ'interno del confine comunale**
- **non si può andare verso i comuni capoluogo"*

caricamento dei confini dei comuni capoluogo confinanti con la Provincia autonoma di Trento

In [19]:
confine_verona = gpd.read_file('data' + os.sep + "verona.geojson")
confine_belluno = gpd.read_file('data' + os.sep + "belluno.geojson")
confine_bolzano = gpd.read_file('data' + os.sep + 'bolzano.geojson')
exclude = []
exclude.append("verona")
exclude.append("belluno")
exclude.append("bolzano")
exclude.append("trento")

**getGeometry(comune_id)**
- unione delle aree dei confini comunali all'area della isodistanza

In [17]:
def getGeometry(comune_id):
    geocomune = comuni_under6000[comuni_under6000.PRO_COM_T == comune_id]
    geocomune30k = gpd.read_file('data' + os.sep + comune_id + ".geojson")
    unionecomune30k = gpd.overlay(geocomune30k, geocomune, how='union').dissolve()
    unionecomune30k['CROSSTN'] = 0
    unionecomune30k['OUTSIDE'] = 0
    unionecomune30k['CROSSBL'] = 0
    unionecomune30k['CROSSBZ'] = 0
    area = unionecomune30k.geometry[unionecomune30k.index[0]]
    trento = confine_trento.geometry[confine_trento.index[0]]
    verona = confine_verona.geometry[confine_verona.index[0]]
    belluno = confine_belluno.geometry[confine_belluno.index[0]]
    bolzano = confine_bolzano.geometry[confine_bolzano.index[0]]
    provincia = confini_pat.geometry[confini_pat.index[0]]
    if (area.intersects(trento)):
        print("removing the borders of Trento")
        unionecomune30k = gpd.overlay(unionecomune30k,confine_trento,how='difference')
        unionecomune30k['CROSSTN'] = 1
    if (area.intersects(verona)):
        print("removing the borders of Verona")
        unionecomune30k = gpd.overlay(unionecomune30k,confine_verona,how='difference')
        unionecomune30k['CROSSVR'] = 1
    if (area.intersects(belluno)):
        print("removing the borders of Belluno")
        unionecomune30k = gpd.overlay(unionecomune30k,confine_belluno,how="difference")
        unionecomune30k['CROSSBL'] = 1
    if (area.intersects(bolzano)):
        print("removing the borders of Bolzano")
        unionecomune30k = gpd.overlay(unionecomune30k,confine_bolzano,how="difference")
        unionecomune30k['CROSSBZ'] = 1
    if (area.intersects(provincia)):
        unionecomune30k['OUTSIDE'] = 1
    return(unionecomune30k[['geometry','COMUNE','PRO_COM_T','CROSSTN','CROSSBL','CROSSBZ']])

## applicazione del calcolo 
i file creati finiscono nella sotto-directory "**area30km**" di "**data**"

In [26]:
for geojson in geojson_files:
    comune_id = geojson.replace(".geojson","")
    print("analysing ... " + comune_id)
    if ((comune_id in exclude) == False):
        filesize = os.stat("data" + os.sep + geojson).st_size
        if filesize > 194:
            newgeom = getGeometry(comune_id)
            filename = areasdata + os.sep + comune_id + ".geojson"
            newgeom.to_file(filename,driver="GeoJSON")
        else:
            print(comune_id)

analysing ... 022009
removing the borders of Trento


In [27]:
print("Finito")

Finito
